## Import stuff

In [6]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import layers
import shutil
import random
from PIL import Image
import numpy as np
import logging
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
logger = tf.get_logger()
logger.setLevel(logging.ERROR)


from tensorflow.python.client import device_lib
device_lib.list_local_devices() 


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13118909115092292733]

## Load training data from csv


In [34]:
csv = pd.read_csv("train.csv")

Y = csv["y"].tolist()
# load columns x1 to x13 as an array of vectors
X = np.array(csv[["x1","x2","x3","x4","x5","x6","x7","x8","x9","x10","x11","x12","x13"]].values)

X = np.array(X,  dtype=float)
Y = np.array(Y,  dtype=float)

# X = tf.cast(X, tf.float32)
# Y = tf.cast(Y, tf.float32)



## Create the model and compile it, with loss and optimizer functions

In [39]:
reg_params = [0.1, 1, 10, 100, 200]
res = np.zeros(5)

for i in range(5):

    reg = reg_params[i]
    all_RMSE = np.zeros(10)

    # Define the K-fold Cross Validator
    kfold = KFold(n_splits=10, shuffle=True)

    # K-fold Cross Validation model evaluation
    fold_no = 0
    for train, test in kfold.split(X, Y):

        # print(train.shape)
        # print(X.shape)
        # print(X[train].shape)
        # print(X[train])
        # print(test.shape)
        
        #,kernel_initializer='zeros'
        model = tf.keras.Sequential([
            layers.Dense(1, input_shape=[13],kernel_regularizer=tf.keras.regularizers.L2(reg),bias_regularizer=tf.keras.regularizers.L2(reg)),
        
        ])

        model.compile(optimizer='Adam', loss='mse') #, metrics=['mse']

        history = model.fit(X[train], Y[train], epochs=50, verbose=False) # 
        

        pred = model.predict(X[test])
        # print(pred)

        RMSE = mean_squared_error(Y[test], pred)**0.5

        
        all_RMSE[fold_no] = RMSE

        fold_no+=1

    print("Finished training for reg = ", reg)
    res[i] = np.mean(all_RMSE)
    # print("RMSE average with param ", reg,": ", np.average(all_RMSE))

print(res)


Finished training for reg =  0.1
Finished training for reg =  1
Finished training for reg =  10
Finished training for reg =  100
Finished training for reg =  200
[ 77.11615415 113.45618585  79.3747164   93.26298362  79.40935049]


# Export results

In [37]:
# write the RMSE to file "submit.csv"
with open("submit.csv", "w") as f:
    for i in res:
        f.write(str(i) + "\n")

